In [1]:
import sys
sys.path.append("..")
import ScraperFC as sfc # import local scraperfc
# import traceback

In [2]:
scraper = sfc.FBRef()
try:
    sfc.shared_functions.check_season(1900, "EPL", "FBRef")
finally:
    scraper.close()

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 106.0.5249
[WDM] - Get LATEST driver version for 106.0.5249
[WDM] - Get LATEST driver version for 106.0.5249
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Owner\.wdm\drivers\chromedriver\win32\106.0.5249.61]


InvalidYearException: 1900 invalid for source FBRef and league EPL. Must be 1993 or later.